In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import csv
import os
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup  #### Scraping library
# !pip install geocoder
import geocoder # import geocoder

print('Libraries imported.')

Libraries imported.


In [2]:
website_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

### Scraping Code

def scrape(url, output_name):
    """Create CSVs from all tables in a Wikipedia article.
    ARGS:
        url (str): The full URL of the Wikipedia article to scrape tables from.
        output_name (str): The base file name (without filepath) to write to.
    """

    # Read tables from Wikipedia article into list of HTML strings
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, "lxml")
    table_classes = {"class": ["sortable", "plainrowheaders"]}
    wikitables = soup.findAll("table", table_classes)

    # Create folder for output if it doesn't exist
    os.makedirs(output_name, exist_ok=True)

    for index, table in enumerate(wikitables):
        # Make a unique file name for each CSV
        if index == 0:
            filename = output_name
        else:
            filename = output_name + "_" + str(index)

        filepath = os.path.join(output_name, filename) + ".csv"

        with open("canada.csv", mode="w", newline="", encoding="utf-8") as output:
            csv_writer = csv.writer(output, quoting=csv.QUOTE_ALL, lineterminator="\n")
            write_html_table_to_csv(table, csv_writer)


def write_html_table_to_csv(table, writer):
    """Write HTML table from Wikipedia to a CSV file.
    ARGS:
        table (bs4.Tag): The bs4 Tag object being analyzed.
        writer (csv.writer): The csv Writer object creating the output.
    """

    # Hold elements that span multiple rows in a list of
    # dictionaries that track 'rows_left' and 'value'
    saved_rowspans = []
    for row in table.findAll("tr"):
        cells = row.findAll(["th", "td"])

        # If the first row, use it to define width of table
        if len(saved_rowspans) == 0:
            saved_rowspans = [None for _ in cells]
        # Insert values from cells that span into this row
        elif len(cells) != len(saved_rowspans):
            for index, rowspan_data in enumerate(saved_rowspans):
                if rowspan_data is not None:
                    # Insert the data from previous row; decrement rows left
                    value = rowspan_data["value"]
                    cells.insert(index, value)

                    if saved_rowspans[index]["rows_left"] == 1:
                        saved_rowspans[index] = None
                    else:
                        saved_rowspans[index]["rows_left"] -= 1

        # If an element with rowspan, save it for future cells
        for index, cell in enumerate(cells):
            if cell.has_attr("rowspan"):
                rowspan_data = {"rows_left": int(cell["rowspan"]), "value": cell}
                saved_rowspans[index] = rowspan_data

        if cells:
            # Clean the data of references and unusual whitespace
            cleaned = clean_data(cells)

            # Fill the row with empty columns if some are missing
            # (Some HTML tables leave final empty cells without a <td> tag)
            columns_missing = len(saved_rowspans) - len(cleaned)
            if columns_missing:
                cleaned += [None] * columns_missing

            writer.writerow(cleaned)
            
def clean_data(row):
    """Clean table row list from Wikipedia into a string for CSV.
    ARGS:
        row (bs4.ResultSet): The bs4 result set being cleaned for output.
    RETURNS:
        cleaned_cells (list[str]): List of cleaned text items in this row.
    """

    cleaned_cells = []

    for cell in row:
        # Strip references from the cell
        references = cell.findAll("sup", {"class": "reference"})
        if references:
            for ref in references:
                ref.extract()

        # Strip sortkeys from the cell
        sortkeys = cell.findAll("span", {"class": "sortkey"})
        if sortkeys:
            for ref in sortkeys:
                ref.extract()

        # Strip footnotes from text and join into a single string
        text_items = cell.findAll(text=True)
        no_footnotes = [text for text in text_items if text[0] != "["]

        cleaned = (
            "".join(no_footnotes)  # Combine elements into single string
            .replace("\xa0", " ")  # Replace non-breaking spaces
            .replace("\n", " ")  # Replace newlines
            .strip()
        )

        cleaned_cells += [cleaned]

    return cleaned_cells

In [3]:
### Save the results in 'canada.csv'

In [4]:
scrape(website_url,'canada')

In [5]:
### Read Data From CSV

In [6]:
data=pd.read_csv("canada.csv",engine='python',encoding='utf-8')
data.head(5)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [7]:
### Remove rows where Borough is not assigned

In [8]:
data=data[data.Borough!='Not assigned']
data.head(5)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [9]:
### Assign Neighbourhood where it is not assigned

In [10]:
data['Neighbourhood']=np.where(data.Neighbourhood=='Not assigned',data.Borough,data.Neighbourhood)
data.head(5)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [11]:
### Print Shape

In [12]:
data.shape

(211, 3)

In [13]:
### Read Lat,Long Data from CSV url

In [14]:
latlongdata=pd.read_csv('http://cocl.us/Geospatial_data')
latlongdata.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
### Join to original data to create updated dataset

In [16]:
data_updated=pd.merge(data,latlongdata,left_on='Postcode',right_on='Postal Code',how='left').drop('Postal Code',axis=1)
data_updated.head(5)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763


In [17]:
### Boroughs with Toronto in name

In [18]:
neighborhoods=data_updated[data_updated.Borough.str.contains('Toronto')]

print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 4 boroughs and 74 neighborhoods.


In [19]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [20]:
### Create map of Toronto using latitude and longitude values

In [21]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

In [22]:
### Add markers to map

In [23]:
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [23]:
neighborhoods.groupby('Borough').agg({'Neighbourhood':'count'})

,Neighbourhood
Borough,
Central Toronto,17
Downtown Toronto,37
East Toronto,7
West Toronto,13


In [24]:
### We Will Only focus on West Toronto Since there are limitations on FourSquare

In [25]:
neighborhoods=neighborhoods[neighborhoods.Borough=='West Toronto']

In [26]:
### Initialize FourSquare

In [27]:
CLIENT_ID = 'KXYXZLQG2SGSBYNTZG3YZVISCO45L3U4ZTXJXVWRV0VHWXUW' # your Foursquare ID
CLIENT_SECRET = 'V3O5JXGIAKA3SRAXRXR1VICGKBGMP4R3XXVRHVQRRHFCVTGT' # your Foursquare Secret
VERSION = '20180604'
radius = 500
LIMIT = 250


print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KXYXZLQG2SGSBYNTZG3YZVISCO45L3U4ZTXJXVWRV0VHWXUW
CLIENT_SECRET:V3O5JXGIAKA3SRAXRXR1VICGKBGMP4R3XXVRHVQRRHFCVTGT


In [28]:
### create dataframes names based on Neighborhoods

In [29]:
neighborhoods['DF_name']='df_'+neighborhoods['Neighbourhood'].apply(lambda x: x.replace(' ','_'))
neighborhoods.reset_index(inplace=True,drop=True)

In [30]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [31]:
###create dataframes for all neighborhoods

In [32]:
for i in range(len(neighborhoods)):
    neighborhood_latitude = neighborhoods.Latitude.iloc[i] # neighborhood latitude value
    neighborhood_longitude = neighborhoods.Longitude.iloc[i] # neighborhood longitude value
    neighborhood_name = neighborhoods.Neighbourhood.iloc[i] # neighborhood name

    #create the correct GET request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        neighborhood_latitude, 
        neighborhood_longitude, 
        radius, 
        LIMIT)
    
    results = requests.get(url).json()

    venues = results['response']['groups'][0]['items']

    temp = json_normalize(venues) # flatten JSON

    # filter columns
    filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
    temp = temp.loc[:, filtered_columns]

    # filter the category for each row
    temp['venue.categories'] = temp.apply(get_category_type, axis=1)

    # clean columns

    temp.columns = [col.split(".")[-1] for col in temp.columns]
    temp.insert(0, 'neighborhood',neighborhoods.Neighbourhood.iloc[i] )
    exec("{}=temp".format(neighborhoods.DF_name.iloc[i]))

    print('{} venues were returned by Foursquare for {}.'.format(temp.shape[0],neighborhoods.Neighbourhood.iloc[i]))

18 venues were returned by Foursquare for Dovercourt Village.
18 venues were returned by Foursquare for Dufferin.
64 venues were returned by Foursquare for Little Portugal.
64 venues were returned by Foursquare for Trinity.
19 venues were returned by Foursquare for Brockton.
19 venues were returned by Foursquare for Exhibition Place.
19 venues were returned by Foursquare for Parkdale Village.
24 venues were returned by Foursquare for High Park.
24 venues were returned by Foursquare for The Junction South.
16 venues were returned by Foursquare for Parkdale.
16 venues were returned by Foursquare for Roncesvalles.
40 venues were returned by Foursquare for Runnymede.
40 venues were returned by Foursquare for Swansea.


In [33]:
q="df_venues=pd.concat(["
for i in neighborhoods.DF_name:
    q=q+i+','
q=q[:-1]+'])'
exec(q)
df_venues['count'] = 1
df_venues.shape
    

(381, 6)

In [34]:
total_venues = pd.pivot_table(df_venues,index=["neighborhood"], values=["count"],aggfunc=np.sum)
total_venues

,count
neighborhood,
Brockton,19
Dovercourt Village,18
Dufferin,18
Exhibition Place,19
High Park,24
Little Portugal,64
Parkdale,16
Parkdale Village,19
Roncesvalles,16


In [36]:
df_venues2 = df_venues.copy()
df_venues3 = df_venues.copy()
df_venues_rest = df_venues2[df_venues2['categories'].str.contains('Restaurant')].reset_index(drop=True)
df_venues_rest['Venue Type'] = 'Restaurant'
df_venues_hotel = df_venues3[df_venues3['categories'].str.contains('Hotel')].reset_index(drop=True)
df_venues_hotel['Venue Type'] = 'Hotel'
df_venues_final = pd.concat([df_venues_rest,df_venues_hotel]).reset_index(drop=True)
df_venues_final.shape

(91, 7)

In [37]:
pivot = pd.pivot_table(df_venues_final,index=["neighborhood","Venue Type"], values=["count"],aggfunc=np.sum)
pivot

,,count
neighborhood,Venue Type,
Brockton,Restaurant,3
Dovercourt Village,Restaurant,1
Dufferin,Restaurant,1
Exhibition Place,Restaurant,3
High Park,Restaurant,6
Little Portugal,Restaurant,21
Parkdale,Restaurant,4
Parkdale Village,Restaurant,3
Roncesvalles,Restaurant,4


In [38]:
df_venues_final.groupby('neighborhood')['Venue Type']\
    .value_counts()\
    .unstack(level=1)\
    .plot.bar(stacked=True)

In [39]:
# one hot encodin
toronto_onehot = pd.get_dummies(df_venues_final[['categories']], prefix="", prefix_sep="")

In [40]:
# add neighborhood column back to dataframe
toronto_onehot['neighborhood'] = df_venues_final['neighborhood'] 

In [41]:
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,neighborhood,Argentinian Restaurant,Asian Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Cuban Restaurant,Eastern European Restaurant,Falafel Restaurant,Fast Food Restaurant,French Restaurant,Greek Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Malay Restaurant,Mexican Restaurant,Middle Eastern Restaurant,New American Restaurant,Polish Restaurant,Restaurant,Southern / Soul Food Restaurant,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,Dovercourt Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,Dufferin,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,Little Portugal,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Little Portugal,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Little Portugal,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [42]:
toronto_onehot = toronto_onehot.groupby('neighborhood').mean().reset_index()
toronto_onehot

,neighborhood,Argentinian Restaurant,Asian Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Cuban Restaurant,Eastern European Restaurant,Falafel Restaurant,Fast Food Restaurant,French Restaurant,Greek Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Malay Restaurant,Mexican Restaurant,Middle Eastern Restaurant,New American Restaurant,Polish Restaurant,Restaurant,Southern / Soul Food Restaurant,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,Brockton,0.000000,0.000000,0.000000,0.333333,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Dovercourt Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Dufferin,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Exhibition Place,0.000000,0.000000,0.000000,0.333333,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,High Park,0.000000,0.000000,0.166667,0.000000,0.000000,0.00,0.000000,0.166667,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.333333,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000
5,Little Portugal,0.047619,0.142857,0.000000,0.000000,0.047619,0.00,0.000000,0.000000,0.047619,0.047619,0.047619,0.047619,0.047619,0.000000,0.047619,0.000000,0.0,0.095238,0.047619,0.095238,0.047619,0.000000,0.047619,0.000000,0.047619,0.095238
6,Parkdale,0.000000,0.000000,0.000000,0.000000,0.250000,0.25,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Parkdale Village,0.000000,0.000000,0.000000,0.333333,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Roncesvalles,0.000000,0.000000,0.000000,0.000000,0.250000,0.25,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Runnymede,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.111111,0.000000,0.111111,0.000000,0.222222,0.000000,0.000000,0.111111,0.000000,0.000000,0.0,0.000000,0.000000,0.111111,0.000000,0.222222,0.000000,0.000000,0.111111,0.000000


In [43]:
### Top 5 Restaurant Types For Each Venus

num_top_venues = 5

for hood in toronto_onehot['neighborhood']:
    print("----"+hood+"----")
    temp = toronto_onehot[toronto_onehot['neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

----Brockton----
                    venue  freq
0    Caribbean Restaurant  0.33
1              Restaurant  0.33
2      Italian Restaurant  0.33
3  Argentinian Restaurant  0.00
4        Malay Restaurant  0.00


----Dovercourt Village----
                           venue  freq
0      Middle Eastern Restaurant   1.0
1         Argentinian Restaurant   0.0
2               Asian Restaurant   0.0
3  Vegetarian / Vegan Restaurant   0.0
4                Thai Restaurant   0.0


----Dufferin----
                           venue  freq
0      Middle Eastern Restaurant   1.0
1         Argentinian Restaurant   0.0
2               Asian Restaurant   0.0
3  Vegetarian / Vegan Restaurant   0.0
4                Thai Restaurant   0.0


----Exhibition Place----
                    venue  freq
0    Caribbean Restaurant  0.33
1              Restaurant  0.33
2      Italian Restaurant  0.33
3  Argentinian Restaurant  0.00
4        Malay Restaurant  0.00


----High Park----
                       venue  freq
0

In [44]:
### Most Recommended Neighborhoods

num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [45]:
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['neighborhood'] = toronto_onehot['neighborhood']

for ind in np.arange(toronto_onehot.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_onehot.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Brockton,Caribbean Restaurant,Restaurant,Italian Restaurant,Vietnamese Restaurant,Japanese Restaurant
1,Dovercourt Village,Middle Eastern Restaurant,Vietnamese Restaurant,Japanese Restaurant,Asian Restaurant,Cajun / Creole Restaurant
2,Dufferin,Middle Eastern Restaurant,Vietnamese Restaurant,Japanese Restaurant,Asian Restaurant,Cajun / Creole Restaurant
3,Exhibition Place,Caribbean Restaurant,Restaurant,Italian Restaurant,Vietnamese Restaurant,Japanese Restaurant
4,High Park,Mexican Restaurant,Thai Restaurant,Cajun / Creole Restaurant,Fast Food Restaurant,Italian Restaurant
5,Little Portugal,Asian Restaurant,Vietnamese Restaurant,Restaurant,New American Restaurant,Malay Restaurant
6,Parkdale,Cuban Restaurant,Restaurant,Eastern European Restaurant,Italian Restaurant,Vietnamese Restaurant
7,Parkdale Village,Caribbean Restaurant,Restaurant,Italian Restaurant,Vietnamese Restaurant,Japanese Restaurant
8,Roncesvalles,Cuban Restaurant,Restaurant,Eastern European Restaurant,Italian Restaurant,Vietnamese Restaurant
9,Runnymede,Italian Restaurant,Sushi Restaurant,Restaurant,Falafel Restaurant,French Restaurant
